In [1]:
import os
import sys
import openai
# two directories up
sys.path.append(os.path.join(os.getcwd(), os.pardir, os.pardir))
from src.db.vector_store import load_config, load_vector_store, OPENAI_API_KEY, QDRANT_API_KEY
from qdrant_client import QdrantClient

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["QDRANT_API_KEY"] = QDRANT_API_KEY

EMBEDDING_MODEL = "text-embedding-3-small"

Qdrant API Key:  f3TPb
OpenAI API Key:  sk-pr


In [2]:
config = load_config()

In [3]:
QDRANT_URL = config.get('qdrant').get('url')
QDRANT_COLLECTION_NAME = config.get('qdrant').get('collection_name')

# This initializes the Qdrant client using the QDRANT_URL and QDRANT_API_KEY
qdrant_client = QdrantClient(
    url=QDRANT_URL,
    api_key=QDRANT_API_KEY
)

In [4]:
from langchain_community.vectorstores import Qdrant

In [5]:
def query_qdrant(query: str, collection_name: str, vector_name:str, top_k:int=5) -> dict:
    """
    Query the Qdrant vector store with a given query and return the top_k results

    Args:
        query (str): The query to search for
        collection_name (str): The name of the collection to search in
        vector_name (str): The name of the vector to search in
        top_k (int): The number of results to return

    Returns:
        dict: The top_k results from the Qdrant vector store
    """

    embedded_query = openai.embeddings.create(
        input=query,
        model=EMBEDDING_MODEL
    ).data[0].embedding

    query_results = qdrant_client.search(
        collection_name=collection_name,
        query_vector=(
            vector_name, embedded_query
        ),
        limit=top_k, 
        query_filter=None
    )

    return query_results

In [15]:
#results = query_qdrant("Method to align structures", QDRANT_COLLECTION_NAME, "content")
#results = query_qdrant("Select the 1a8o structure", QDRANT_COLLECTION_NAME, "content")
results = query_qdrant("how to select atoms", QDRANT_COLLECTION_NAME, "content")#

In [16]:
for i, article in enumerate(results):
    print(f"{i+1}. score: {article.score} - Function: {article.payload['name']}")
    print(f"{article.payload['content'].strip('')}\n")

1. score: 0.5422758 - Function: Select
Select select creates a named selection from an atom selection.  Selections are one of the most powerful aspects of PyMOL and learning to use selections well is paramount to quickly achieving your goals in PyMOL.

Contents

1 Usage
2 Arguments
3 Examples
4 Notes
5 See Also


Type help selections for more information about selections. 

See Also
Selection Algebra
Property Selectors




Retrieved from "https://pymolwiki.org/index.php?title=Select&oldid=12779"
Categories: Pages using deprecated source tagsCommandsSelecting

 name = str: a unique name for the selection {default if skipped: sele}
selection = str: a selection-expression
enable = 0/1: show selection indicators {default: 1}
quiet = 0/1 {default: 0 for command, 1 for API}
merge = 0/1: update existing named selection {default: 0}
state = int: object state, affects spacial operators like within {default: 0 (all states)}
 select near , (ll expand 8)
select near , (ll expand 8)
select bb, (nam

In [8]:
from langchain_openai import OpenAIEmbeddings

embed_model = OpenAIEmbeddings(model=EMBEDDING_MODEL)

In [9]:
from langchain_qdrant import Qdrant

qdrant = Qdrant.from_existing_collection(
    url=QDRANT_URL,
    embedding=embed_model,
    collection_name=QDRANT_COLLECTION_NAME,
    vector_name="content", 
    content_payload_key='content',
    distance_strategy='COSINE'
)

In [10]:
query = "Method to align structures"
results = qdrant.similarity_search(query=query, k=3)

In [11]:
# the context will be the content of each document
context = ""

for i, result in enumerate(results):
    context += f"{i+1}. Document:\n{result.page_content}\n\n"

In [12]:
from langchain.schema import (
    SystemMessage,    # Generic system message
    HumanMessage,     # HumanMessage is a message from the human
    AIMessage,        # AIMessage is a message from the AI
)

messages = [SystemMessage(content="This is a conversation between a human and an AI assistant.")]

def generate_grader_prompt(query: str, qdrant: Qdrant, top_k:int=3) -> HumanMessage:
    LLM_GRADER_PROMPT="""
    Given a query and context document retrieved from a database, 
    grade each document based on if it is relevant to the query
    as either 'relevant' or 'not relevant'.

    Query: {query}
    Context: {context}
    """
    results = qdrant.similarity_search(query=query, k=top_k)

    # the context will be the content of each document
    context = ""

    for i, result in enumerate(results):
        context += f"{i+1}. Document:\n{result.page_content}\n\n"

    prompt = HumanMessage(content=LLM_GRADER_PROMPT.format(
        query=query,
        context=context
    ))

    return prompt

messages.append(generate_grader_prompt(query=query, qdrant=qdrant, top_k=3))

res = chat.



SyntaxError: invalid syntax (1606323524.py, line 35)